In [ ]:
import os
import open3d as o3d
import numpy as np
from tqdm import tqdm

# load proteins in the form of point cloud xyz format 
# and returns train and validation data according to validation_rate

# load protein dataset
input_directory = './protein_shape'

def load_proteins (input_directory, validation_rate):

    train_points = []
    test_points = []
    
    file_list = os.listdir(input_directory)
    
    for file_name in tqdm(file_list):
        
        # read point cloud
        full_path = os.path.join(input_directory, file_name)
        pcd = o3d.io.read_point_cloud(full_path)
        points = np.asarray(pcd.points)
        
        # add to list
        train_points.append(points)
    
    
    train_points = np.array(train_points)
    validation_length = (int)(train_points.shape[0]*validation_rate)
    train_length = train_points.shape[0]- validation_length
    np.random.shuffle(train_points)
    
    train_points, test_points = train_points[:train_length,:,:], train_points[train_length:,:,:]
    
    return train_points, train_points, test_points, test_points


train_points, train_labels, test_points, test_labels = load_proteins(input_directory, 0.2)

print(train_points.shape)
print(train_labels.shape)

print(test_points.shape)
print(test_labels.shape)

    

In [ ]:
# This code cell is adapted from https://keras.io/examples/vision/pointnet/
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers


#tf.random.set_random_seed(1234)

def augment(points, labels):
    # jitter points
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    # shuffle points maybe it is not necessary
    points = tf.random.shuffle(points)
    labels = points
    return points, labels


def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("tanh")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("tanh")(x)

class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))
    
def tnet(inputs, num_features):
    
    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)
    
    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])


NUM_POINTS = 2048
BATCH_SIZE = 16

train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))

# data augmentation
train_dataset = train_dataset.shuffle(len(train_points)).map(augment).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)


inputs = keras.Input(shape=(NUM_POINTS, 3))
x = tnet(inputs, 3)

model = keras.Model(inputs=inputs, outputs=x, name="pointnet")
model.summary()


In [ ]:
# Run this cell to train a new model or skip it to use a model already trained
model.compile(
    loss="cosine_similarity",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint('./pointnet_weights.hdf5',
                                   monitor='val_loss', save_weights_only=True, save_best_only=True)

model.fit(train_dataset, epochs=50, validation_data=test_dataset, callbacks=[model_checkpoint])

In [ ]:
# Run this cell to test the perfomance of the model
# Specify the model to use
model.load_weights('./pointnet_weights.hdf5')
model.compile(
    loss="cosine_similarity",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

model.evaluate(test_points, test_labels)

In [ ]:
# Use the trained model to extract features
model_feature_512 = keras.Model(inputs=model.input, outputs=model.get_layer('global_max_pooling1d').output)
model_feature_256 = keras.Model(inputs=model.input, outputs=model.get_layer('batch_normalization_3').output)
model_feature_128 = keras.Model(inputs=model.input, outputs=model.get_layer('batch_normalization_4').output)

In [ ]:
# read query proteins one by one and transform in features 512, 256, 128, save them in txt file
input_directory = './queries_protein_shape'

file_list = os.listdir(input_directory)

file_list= sorted(file_list, key=lambda x: int(x[:-10]))

list_points =[]

for file_name in tqdm(file_list):
    #print(file_name)
    # read point cloud
    full_path = os.path.join(input_directory, file_name)
    pcd = o3d.io.read_point_cloud(full_path)
    points = np.asarray(pcd.points)
        
    # add to list
    list_points.append(points)
    

proteins = np.array(list_points)
print(proteins.shape)


#extract features
queries_features_128 = model_feature_128.predict(proteins)
print(queries_features_128.shape)

queries_features_256 = model_feature_256.predict(proteins)
print(queries_features_256.shape)

queries_features_512 = model_feature_512.predict(proteins)
print(queries_features_512.shape)

#save queries features to txt files
np.savetxt('./queries_features_128.txt', queries_features_128)
np.savetxt('./queries_features_256.txt', queries_features_256)
np.savetxt('./queries_features_512.txt', queries_features_512)

In [ ]:
# read dataset proteins one by one and transform in features 512, 256, 128, save them in txt file
input_directory = './protein_shape'

file_list = os.listdir(input_directory)

file_list= sorted(file_list, key=lambda x: int(x[:-10]))

list_points =[]

for file_name in tqdm(file_list):
    #print(file_name)
    # read point cloud
    full_path = os.path.join(input_directory, file_name)
    pcd = o3d.io.read_point_cloud(full_path)
    points = np.asarray(pcd.points)
        
    # add to list
    list_points.append(points)
    

proteins = np.array(list_points)
print(proteins.shape)


#extract features
features_128 = model_feature_128.predict(proteins)
print(features_128.shape)

features_256 = model_feature_256.predict(proteins)
print(features_256.shape)

features_512 = model_feature_512.predict(proteins)
print(features_512.shape)

#save dataset features to txt files
np.savetxt('./features_128.txt', features_128)
np.savetxt('./features_256.txt', features_256)
np.savetxt('./features_512.txt', features_512)

In [ ]:
#calculate dissimilarity matrices using euclidean distances
# for the ten queries to the 554 shapes
# and save to binary file
def dissimimarity_function(queries_features, features):
    
    dissim = []
    
    for i in range(queries_features.shape[0]):
        
        dist = (features - queries_features[i])**2
        dist = np.sum(dist, axis=1)
        dist = np.sqrt(dist)
        #dist = dist.reshape(1,554)
        # append distance of query i to all 554 proteins
        dissim.append(dist)
        
    dissim = np.array(dissim)
    return dissim
    

dissim_128 = dissimimarity_function(queries_features_128, features_128)
#print(dissim_128)

dissim_256 = dissimimarity_function(queries_features_256, features_256)
#print(dissim_256)

dissim_512 = dissimimarity_function(queries_features_512, features_512)
#print(dissim_512)


# save binary format
dissim_128.tofile('./dissim_128.bin')
dissim_256.tofile('./dissim_256.bin')
dissim_512.tofile('./dissim_512.bin')

# save txt format
# np.savetxt('dissim_512.txt', dissim_128)
# np.savetxt('dissim_512.txt', dissim_256)
# np.savetxt('dissim_512.txt', dissim_512)

In [ ]:
# load dissimilarity matrices
dissim_512 = np.fromfile('./dissim_512.bin', dtype=np.float32) 
dissim_512 = dissim_512.reshape(10, 554)
print(dissim_512)

dissim_256 = np.fromfile('./dissim_256.bin', dtype=np.float32) 
dissim_256 = dissim_256.reshape(10, 554)
print(dissim_256)

dissim_128 = np.fromfile('./dissim_128.bin', dtype=np.float32) 
dissim_128 = dissim_128.reshape(10, 554)
print(dissim_128)